# 02 - Level 2: Ray 并发任务

本教程演示 Level 2 部署：引入 Ray 作为执行后端，支持并发任务。

## Level 1 vs Level 2

| 维度 | Level 1 | Level 2 |
|------|---------|--------|
| 执行后端 | LocalRunner（线程） | RayRunner（Ray Task） |
| 并发 | 串行，一个任务跑完才能跑下一个 | 并发，多个 Ray Task 同时执行 |
| Daft 后端 | native runner（本地执行） | native runner（Ray Worker 内本地执行） |
| 切换方式 | `PLATFORM_LEVEL=1`（默认） | `PLATFORM_LEVEL=2` |

## 前置条件

```bash
pip install -r requirements.txt
pip install ray  # Level 2 额外依赖
```

**注意**：需要先跑过 Level 1 教程，确保 `.ai_platform/datasets/mnist_clean.lance` 已存在。

## 1. 启动 Level 2 Server

唯一的区别是设置环境变量 `PLATFORM_LEVEL=2`，Server 会自动使用 RayRunner。

In [ ]:
import os
import subprocess
import time

import httpx

# notebook 的 cwd 是 notebooks/，需要切到上级目录启动服务
PROJECT_DIR = os.path.dirname(os.getcwd())

# 设置 Level 2 环境变量
env = os.environ.copy()
env["PLATFORM_LEVEL"] = "2"

# 启动 AI Platform Server（Level 2）
server_proc = subprocess.Popen(
    ["uvicorn", "ai_platform.app:app", "--port", "8000"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    cwd=PROJECT_DIR,
    env=env,
)
print(f"Server 已启动 (PID: {server_proc.pid}, PLATFORM_LEVEL=2)")

# 等待服务就绪（Ray 初始化较慢，轮询等待）
BASE_URL = "http://localhost:8000/api/v1"

for i in range(30):
    if server_proc.poll() is not None:
        print(f"[ERROR] 进程已退出，退出码: {server_proc.returncode}")
        print(f"stderr: {server_proc.stderr.read().decode()}")
        break
    try:
        resp = httpx.get(f"{BASE_URL}/datasets", timeout=2)
        if resp.status_code == 200:
            print(f"服务就绪（等待 {i + 1}s）")
            print(f"数据集: {[d['id'] for d in resp.json()]}")
            break
    except (httpx.ConnectError, httpx.ReadTimeout):
        pass
    time.sleep(1)
else:
    print("[ERROR] 服务启动超时（30s）")

## 2. 并发训练：同时提交多个任务

Level 1 下任务串行执行，提交第二个任务要等第一个跑完。

Level 2 下任务提交为 Ray Task，可以并发执行。我们同时提交两个训练任务，用不同的超参数：

In [ ]:
# 同时提交两个训练任务，不同超参数
configs = [
    {"name": "mnist_cnn_lr001", "params": {"epochs": 2, "learning_rate": 0.001, "batch_size": 64, "device": "cpu"}},
    {"name": "mnist_cnn_lr01",  "params": {"epochs": 2, "learning_rate": 0.01,  "batch_size": 64, "device": "cpu"}},
]

tasks = []
for cfg in configs:
    resp = httpx.post(f"{BASE_URL}/tasks", json={
        "name": cfg["name"],
        "input": ".ai_platform/datasets/mnist_clean.lance",
        "script": "mnist/mnist_cnn.py",
        "params": cfg["params"],
        "output": f".ai_platform/models/{cfg['name']}.lance",
    })
    task = resp.json()
    tasks.append(task)
    print(f"已提交: {task['id']} ({cfg['name']})")

print(f"\n两个任务同时在 Ray 上执行...")

In [ ]:
# 轮询等待所有任务完成
pending = {t["id"]: t["name"] for t in tasks}
results = {}

while pending:
    for task_id in list(pending.keys()):
        resp = httpx.get(f"{BASE_URL}/tasks/{task_id}")
        task = resp.json()
        if task["status"] in ("completed", "failed"):
            name = pending.pop(task_id)
            results[name] = task
            print(f"{name}: {task['status']}")
    if pending:
        time.sleep(5)

# 对比结果
print("\n--- 超参数对比 ---")
for name, task in results.items():
    if task["status"] == "completed":
        r = task["result"]
        print(f"{name}: accuracy={r['accuracy']:.2%}, loss={r['test_loss']:.4f}")
    else:
        print(f"{name}: 失败 - {task.get('error')}")

## 3. 可视化对比

对比两组超参数的训练结果：

In [ ]:
import matplotlib.pyplot as plt

names = []
accuracies = []
losses = []

for name, task in results.items():
    if task["status"] == "completed":
        names.append(name)
        accuracies.append(task["result"]["accuracy"])
        losses.append(task["result"]["test_loss"])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

ax1.bar(names, accuracies, color=["steelblue", "coral"])
ax1.set_ylabel("Accuracy")
ax1.set_title("Test Accuracy")
ax1.set_ylim(0.9, 1.0)

ax2.bar(names, losses, color=["steelblue", "coral"])
ax2.set_ylabel("Loss")
ax2.set_title("Test Loss")

plt.suptitle("Level 2: Concurrent Hyperparameter Comparison", fontsize=14)
plt.tight_layout()
plt.show()

## 4. 查看 Ray 集群状态

Level 2 下 Ray 以本地集群模式运行，可以查看集群资源：

In [ ]:
import ray

if not ray.is_initialized():
    ray.init(address="auto", ignore_reinit_error=True)

resources = ray.cluster_resources()
print("Ray 集群资源:")
for k, v in sorted(resources.items()):
    print(f"  {k}: {v}")

## 5. 清理

In [ ]:
server_proc.terminate()
server_proc.wait()
print("Server 已停止")

## 总结

Level 2 的核心变化：

| 维度 | 改动 |
|------|------|
| 环境变量 | `PLATFORM_LEVEL=2` |
| 执行后端 | LocalRunner → RayRunner（Ray Task） |
| 并发能力 | 串行 → 多个 Ray Task 并发 |

**用户脚本不需要任何改动**——同样的 `mnist_cnn.py`，在 Level 1 跑线程，在 Level 2 跑 Ray Task。这就是 Runner 抽象的价值。

### 进阶方向

- Level 3: Ray on K8s + S3 共享存储，多机多任务
- Daft-on-Ray 分布式执行图适合大数据量场景（Level 3）
- 详见 [README.md](../ai_platform/README.md) 中的部署级别设计